In [1]:
import pandas as pd 
import numpy as np #maybe delete
import re
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def getFirstName(df, nameColumn): #nameColumn as list
    firstName = []
    for n in df[nameColumn]:
        nameSplit = re.split(' ', n)
        firstName.append(nameSplit[0])
    return firstName

In [3]:
def assignGender(df, NameCol):
    index = 0
    nameDict = {}
    genderEstimation = pd.read_csv("Files/wgnd_2_0_name-gender-code.csv") #genderEstimation file
    genderEstimation['Name'] = genderEstimation.name.str.replace(r'"|\'', '', regex=True).str.lower() #remove " and ' and make to lower case
  
    usNames = genderEstimation[genderEstimation.code == 'US'] #some names can't not be found, but can with another code
    otherNames = genderEstimation[~genderEstimation.Name.isin(usNames.Name)] #names not in US

    uniqueNamesUS = df[df[NameCol].isin(usNames.Name)][NameCol].unique() #df with names from presenters, that is also in genderEstimation
    uniqueNamesOther = df[df[NameCol].isin(otherNames.Name)][NameCol].unique()
    
    for n in tqdm(uniqueNamesUS): #loop through us names that is also in the presenter df
        nameTable = usNames[usNames.Name == n] #table with the name 
        genderIndex = nameTable.wgt.argmax() #index of gender with highest score/probability
        gender = nameTable.iloc[genderIndex].gender #gender that is most likely
        nameDict[index] = {'firstName': n, 'Gender': gender} #assign gender to the name in a dict
        index += 1
    
    for m in tqdm(uniqueNamesOther): #loop through other names that is also in the presenter df
        nameTable = otherNames[otherNames.Name == m].groupby('gender').mean() #table with the name - mean of wgt
        genderIndex = nameTable.wgt.argmax() #index of gender with higest average probability
        gender = nameTable.iloc[genderIndex].name
        nameDict[index] = {'firstName': m, 'Gender': gender} #assign gender to the name in a dict
        index += 1
    
    return nameDict

#### Add gender

In [4]:
#load data
researchers = pd.read_pickle("Files/DataFrames/apiResearchers_fewNameCol.pkl")

In [5]:
#assign gender
genderDict = assignGender(researchers, 'FirstName')

100%|███████████████████████████████████████| 9416/9416 [24:01<00:00,  6.53it/s]


In [6]:
#dataframe witht the gender and name
genderAssign = pd.DataFrame.from_dict(genderDict, orient = 'index')

Now we are merging genderAssign and researchers. This will automatically add the gender to names in the presenters df.

In [7]:
#merge researchers df with gender df
researchers = researchers.merge(genderAssign, how = 'left', left_on = 'FirstName', right_on = 'firstName')

In [8]:
#only keep researchers assigned to either F or M (and not NaN or ?)
researchers_wGender = researchers[(researchers.Gender == 'F') | (researchers.Gender == 'M')]

In [9]:
researchers_wGender.to_pickle("Files/Dataframes/researchers_wGender.pkl") 